## Оптимизация выполнения кода, векторизация, Numba

Материалы:
* Макрушин С.В. Лекция 3: Оптимизация выполнения кода, векторизация, Numba
* IPython Cookbook, Second Edition (2018), глава 4
* https://numba.pydata.org/numba-doc/latest/user/5minguide.html

## Задачи для совместного разбора

1. Сгенерируйте массив `A` из `N=1млн` случайных целых чисел на отрезке от 0 до 1000. Пусть `B[i] = A[i] + 100`. Посчитайте среднее значение массива `B`.

In [70]:
import numpy as np
import pandas as pd
from numba import jit

In [71]:
%load_ext line_profiler

The line_profiler extension is already loaded. To reload it, use:
  %reload_ext line_profiler


In [72]:
arr_A = np.random.randint(0, 1000, size=1_000_000)

In [73]:
%%timeit

sum(arr_A) / len(arr_A) + 100

75.3 ms ± 11.3 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [74]:
%%timeit

np.mean(arr_A) + 100

1.05 ms ± 22.4 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In [75]:
@jit(nopython=True)
def numba_speed_up(arr):
    return sum(arr) / len(arr) + 100


%timeit numba_speed_up(arr_A)

609 µs ± 20.1 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


2. Создайте таблицу 2млн строк и с 4 столбцами, заполненными случайными числами. Добавьте столбец `key`, которые содержит элементы из множества английских букв. Выберите из таблицы подмножество строк, для которых в столбце `key` указаны первые 5 английских букв.

In [76]:
from string import ascii_lowercase

n = 2_000_000
df = pd.DataFrame(np.random.random(size=(n, 4)), columns=[f'random_{i}' for i in range(4)])
df['key'] = np.random.default_rng().choice(list(ascii_lowercase), size=n)
df

,random_0,random_1,random_2,random_3,key
0,0.939471,0.878125,0.182108,0.651388,r
1,0.546216,0.876699,0.069972,0.745263,s
2,0.499358,0.547925,0.675378,0.534381,j
3,0.114853,0.897752,0.134852,0.742542,e
4,0.687868,0.053911,0.848437,0.470144,z
...,...,...,...,...,...
1999995,0.098768,0.417570,0.924354,0.173394,o
1999996,0.493002,0.207158,0.112251,0.614146,y
1999997,0.743497,0.842766,0.979029,0.479115,q
1999998,0.300145,0.445843,0.481387,0.810915,x


In [77]:
def bad_solution(df_):
    letters = ['a', 'b', 'c', 'd', 'e']
    dfs = []
    for letter in letters:
        q = df_[df_['key'] == letter]
        dfs.append(q)
    return pd.concat(dfs, axis=0)


%timeit bad_solution(df)

676 ms ± 28.5 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [78]:
def inline_solution(df_):
    return df_[df_['key'].isin({'a', 'b', 'c', 'd', 'e'})]


%timeit inline_solution(df)

117 ms ± 3.75 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [79]:
def for_profiling_solution(df_):
    letters = {'a', 'b', 'c', 'd', 'e'}
    mask = df_['key'].isin(letters)
    return df_[mask]


%lprun -f for_profiling_solution for_profiling_solution(df)

## Лабораторная работа 3

1. В файлах `recipes_sample.csv` и `reviews_sample.csv` (__ЛР 2__) находится информация об рецептах блюд и отзывах на эти рецепты соответственно. Загрузите данные из файлов в виде `pd.DataFrame` с названиями `recipes` и `reviews`. Обратите внимание на корректное считывание столбца(ов) с индексами. Приведите столбцы к нужным типам.

Реализуйте несколько вариантов функции подсчета среднего значения столбца `rating` из таблицы `reviews` для отзывов, оставленных в 2010 году.

A. С использованием метода `DataFrame.iterrows` исходной таблицы;

Б. С использованием метода `DataFrame.iterrows` таблицы, в которой сохранены только отзывы за 2010 год;

В. С использованием метода `Series.mean`.

Проверьте, что результаты работы всех написанных функций корректны и совпадают. Измерьте выполнения всех написанных функций.


In [80]:
recipes_df = pd.read_csv('sem/data/recipes_sample.csv', sep=',', parse_dates=['submitted'])

recipes_df.info()
recipes_df

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 30000 entries, 0 to 29999
Data columns (total 8 columns):
 #   Column          Non-Null Count  Dtype         
---  ------          --------------  -----         
 0   name            30000 non-null  object        
 1   id              30000 non-null  int64         
 2   minutes         30000 non-null  int64         
 3   contributor_id  30000 non-null  int64         
 4   submitted       30000 non-null  datetime64[ns]
 5   n_steps         18810 non-null  float64       
 6   description     29377 non-null  object        
 7   n_ingredients   21120 non-null  float64       
dtypes: datetime64[ns](1), float64(2), int64(3), object(2)
memory usage: 1.8+ MB


,name,id,minutes,contributor_id,submitted,n_steps,description,n_ingredients
0,george s at the cove black bean soup,44123,90,35193,2002-10-25,NaN,an original recipe created by chef scott meska...,18.0
1,healthy for them yogurt popsicles,67664,10,91970,2003-07-26,NaN,my children and their friends ask for my homem...,NaN
2,i can t believe it s spinach,38798,30,1533,2002-08-29,NaN,"these were so go, it surprised even me.",8.0
3,italian gut busters,35173,45,22724,2002-07-27,NaN,my sister-in-law made these for us at a family...,NaN
4,love is in the air beef fondue sauces,84797,25,4470,2004-02-23,4.0,i think a fondue is a very romantic casual din...,NaN
...,...,...,...,...,...,...,...,...
29995,zurie s holey rustic olive and cheddar bread,267661,80,200862,2007-11-25,16.0,this is based on a french recipe but i changed...,10.0
29996,zwetschgenkuchen bavarian plum cake,386977,240,177443,2009-08-24,NaN,"this is a traditional fresh plum cake, thought...",11.0
29997,zwiebelkuchen southwest german onion cake,103312,75,161745,2004-11-03,NaN,this is a traditional late summer early fall s...,NaN
29998,zydeco soup,486161,60,227978,2012-08-29,NaN,this is a delicious soup that i originally fou...,NaN


In [81]:
reviews_df = pd.read_csv('sem/data/reviews_sample.csv', sep=',', index_col=0, parse_dates=['date'])

reviews_df.info()
reviews_df

<class 'pandas.core.frame.DataFrame'>
Int64Index: 126696 entries, 370476 to 691207
Data columns (total 5 columns):
 #   Column     Non-Null Count   Dtype         
---  ------     --------------   -----         
 0   user_id    126696 non-null  int64         
 1   recipe_id  126696 non-null  int64         
 2   date       126696 non-null  datetime64[ns]
 3   rating     126696 non-null  int64         
 4   review     126679 non-null  object        
dtypes: datetime64[ns](1), int64(3), object(1)
memory usage: 5.8+ MB


,user_id,recipe_id,date,rating,review
370476,21752,57993,2003-05-01,5,Last week whole sides of frozen salmon fillet ...
624300,431813,142201,2007-09-16,5,So simple and so tasty! I used a yellow capsi...
187037,400708,252013,2008-01-10,4,"Very nice breakfast HH, easy to make and yummy..."
706134,2001852463,404716,2017-12-11,5,These are a favorite for the holidays and so e...
312179,95810,129396,2008-03-14,5,Excellent soup! The tomato flavor is just gre...
...,...,...,...,...,...
1013457,1270706,335534,2009-05-17,4,This recipe was great! I made it last night. I...
158736,2282344,8701,2012-06-03,0,This recipe is outstanding. I followed the rec...
1059834,689540,222001,2008-04-08,5,"Well, we were not a crowd but it was a fabulou..."
453285,2000242659,354979,2015-06-02,5,I have been a steak eater and dedicated BBQ gr...


In [82]:
def a_solution(reviews: pd.DataFrame):
    sum_, n_ = 0, 0
    for i, row in reviews.iterrows():
        if row['date'].year == 2010:
            sum_ += row['rating']
            n_ += 1

    return sum_ / n_


%timeit a_solution(reviews_df)

5.44 s ± 82.7 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [83]:
def b_solution(reviews: pd.DataFrame):
    reviews = reviews[reviews['date'].dt.year == 2010]

    sum_, n_ = 0, 0
    for i, row in reviews.iterrows():
        sum_ += row['rating']
        n_ += 1

    return sum_ / n_


%timeit b_solution(reviews_df)

527 ms ± 10.6 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [84]:
def c_solution(reviews: pd.DataFrame):
    return reviews[reviews['date'].dt.year == 2010]['rating'].mean()


%timeit c_solution(reviews_df)

13 ms ± 217 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [85]:
a_result = a_solution(reviews_df)
b_result = b_solution(reviews_df)
c_result = c_solution(reviews_df)

In [86]:
assert a_result == b_result == c_result

2. Какая из созданных функций выполняется медленнее? Что наиболее сильно влияет на скорость выполнения? Для ответа использовать профайлер `line_profiler`. Сохраните результаты работы профайлера в отдельную текстовую ячейку и прокомментируйте результаты его работы.

(*). Сможете ли вы ускорить работу функции 1Б, отказавшись от использования метода `iterrows`, но не используя метод `mean`?

In [87]:
# %lprun -f a_solution a_solution(reviews_df)

Line #      Hits         Time  Per Hit   % Time  Line Contents
==============================================================
     1
     2         1         36.0     36.0      0.0
     3    126697  120483192.0    951.0     88.4
     4    126696   14671916.0    115.8     10.8
     5     12094    1115105.0     92.2      0.8
     6     12094      59278.0      4.9      0.0
     7
     8         1         13.0     13.0      0.0

In [88]:
def b_solution_speed_up(reviews: pd.DataFrame):
    def mean(x: pd.Series) -> float:
        return np.sum(x) / x.size

    reviews = reviews[reviews['date'].dt.year == 2010]
    return mean(reviews['rating'])


%timeit b_solution_speed_up(reviews_df)

13 ms ± 239 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [89]:
b_speed_up_result = b_solution_speed_up(reviews_df)

In [90]:
assert b_speed_up_result == a_result == b_result == c_result

3. Вам предлагается воспользоваться функцией, которая собирает статистику о том, сколько отзывов содержат то или иное слово. Измерьте время выполнения этой функции. Сможете ли вы найти узкие места в коде, используя профайлер? Выпишите (словами), что в имеющемся коде реализовано неоптимально. Оптимизируйте функцию и добейтесь значительного (как минимум, на один порядок) прироста в скорости выполнения.

In [91]:
def get_word_reviews_count(df):  # по-хорошему - очистить текст, привести в нижний регистр  # noqa
    word_reviews = {}  # это позволит получить правильное решение
    for _, row in df.dropna(subset=['review']).iterrows():
        recipe_id, review = row['recipe_id'], row['review']
        words = review.split(' ')
        for word in words:
            if word not in word_reviews:  # defaultdict избавит от if на каждой итерации (чуть быстрее)
                word_reviews[word] = []
            word_reviews[word].append(recipe_id)  # defaultdict(int) можно сразу читать: += 1

    word_reviews_count = {}  # полностью не нужная часть. Мы же уже знаем все слова
    for _, row in df.dropna(subset=['review']).iterrows():
        review = row['review']
        words = review.split(' ')
        for word in words:
            word_reviews_count[word] = len(word_reviews[word])
    return word_reviews_count


%timeit get_word_reviews_count(reviews_df)

18.1 s ± 696 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [92]:
# %lprun -f get_word_reviews_count get_word_reviews_count(reviews_df)

Line #      Hits         Time  Per Hit   % Time  Line Contents
==============================================================
     1
     2         1         15.0     15.0      0.0
     3    126680  135850844.0   1072.4     27.2
     4    126679   27141559.0    214.3      5.4
     5    126679    5479764.0     43.3      1.1
     6   6918689   23000764.0      3.3      4.6
     7   6792010   32686572.0      4.8      6.5
     8    174944     958313.0      5.5      0.2
     9   6792010   36556558.0      5.4      7.3
    10
    11         1          8.0      8.0      0.0
    12    126680  137631290.0   1086.4     27.6
    13    126679   17134982.0    135.3      3.4
    14    126679    5771328.0     45.6      1.2
    15   6918689   24407392.0      3.5      4.9
    16   6792010   52570028.0      7.7     10.5
    17         1         11.0     11.0      0.0

In [93]:
from collections import defaultdict


def little_better_solution(reviews: pd.DataFrame) -> dict:
    word_reviews_count = defaultdict(int)
    for _, row in reviews.dropna(subset=['review']).iterrows():  # далее нужно менять концепцию:
        words = row['review'].split(' ')
        for word in words:  # больше pandas & меньше циклов
            word_reviews_count[word] += 1

    return word_reviews_count


%timeit little_better_solution(reviews_df)

8.35 s ± 213 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [94]:
# %lprun -f little_better_solution little_better_solution(reviews_df)

Line #      Hits         Time  Per Hit   % Time  Line Contents
==============================================================
     4
     5         1         27.0     27.0      0.0
     6    126680  132307567.0   1044.4     57.5
     7    126679   27228255.0    214.9     11.8
     8    126679    5914574.0     46.7      2.6
     9   6918689   23352366.0      3.4     10.1
    10   6792010   41315608.0      6.1     18.0
    11
    12         1          7.0      7.0      0.0

In [95]:
def more_then_better_solution(reviews: pd.DataFrame) -> dict:
    word_reviews_count = defaultdict(int)

    def foo(x):  # bottleneck
        for word in x.split(' '):
            word_reviews_count[word] += 1  # интересно: operator.iadd не работает в таком сценарии

    reviews = reviews['review'].dropna()
    reviews.apply(foo)
    return word_reviews_count


%timeit more_then_better_solution(reviews_df)  # а вот и "как минимум, один порядок"

1.79 s ± 80.8 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [96]:
# %lprun -f more_then_better_solution more_then_better_solution(reviews_df)

Line #      Hits         Time  Per Hit   % Time  Line Contents
==============================================================
     1
     2         1         23.0     23.0      0.0
     3
     4         1         11.0     11.0      0.0
     5
     6
     7
     8         1     112535.0 112535.0      0.3
     9         1   39332455.0 39332455.0     99.7
    10         1         15.0     15.0      0.0

In [97]:
# import pandas as pd
# import modin.pandas as mpd
# import swifter  # noqa
#
#
# def to_the_moon_solution(reviews: mpd.DataFrame) -> dict:
#     word_reviews_count = defaultdict(int)
#
#     def foo(x):  # bottleneck
#         for word in x.split(' '):
#             word_reviews_count[word] += 1
#
#     reviews = reviews['review'].dropna()
#     reviews.swifter.apply(foo)
#     return word_reviews_count
#
#
# modin_reviews_df = mpd.DataFrame(reviews_df)
# %timeit to_the_moon_solution(modin_reviews_df)

In [98]:
# %lprun -f more_then_better_solution more_then_better_solution(reviews_df)

Line #      Hits         Time  Per Hit   % Time  Line Contents
==============================================================
     1
     2         1         40.0     40.0      0.0
     3
     4         1         16.0     16.0      0.0
     5
     6
     7
     8         1    1969667.0 1969667.0      4.0
     9         1   47747783.0 47747783.0     96.0
    10         1         18.0     18.0      0.0

In [99]:
result = get_word_reviews_count(reviews_df)

In [100]:
little_better_result = little_better_solution(reviews_df)

In [101]:
more_then_better_result = more_then_better_solution(reviews_df)

In [102]:
# to_the_moon_result = to_the_moon_solution(mpd.DataFrame(reviews_df))

In [103]:
assert more_then_better_result == little_better_result == result

4. Напишите несколько версий функции `MAPE` (см. [MAPE](https://en.wikipedia.org/wiki/Mean_absolute_percentage_error)) для расчета среднего абсолютного процентного отклонения значения рейтинга отзыва на рецепт от среднего значения рейтинга по всем отзывам для этого рецепта. 
    1. Без использования векторизованных операций и методов массивов `numpy` и без использования `numba`
    2. Без использования векторизованных операций и методов массивов `numpy`, но с использованием `numba`
    3. С использованием векторизованных операций и методов массивов `numpy`, но без использования `numba`
    4. C использованием векторизованных операций и методов массивов `numpy` и `numba`
    
Измерьте время выполнения каждой из реализаций.

Замечание: удалите из выборки отзывы с нулевым рейтингом.


In [104]:
sampling_df = reviews_df[['recipe_id', 'rating']].dropna()
sampling_df = sampling_df[sampling_df['rating'] != 0]
grouped_df = sampling_df.groupby(['recipe_id'])['rating']

In [105]:
def mape_1(actual, forecast) -> float:
    return sum([abs(i - forecast) / i for i in actual]) / len(actual) * 100


def solution_1(grouped: pd.DataFrame) -> pd.Series:
    def wrapper(s: pd.Series) -> float:  # noqa
        return mape_1(s.to_numpy(), s.mean())

    return grouped.agg(wrapper)


%timeit solution_1(grouped_df)

2.2 s ± 27.8 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [106]:
@jit(nopython=True)
def mape_2(actual, forecast) -> float:
    return sum([abs(i - forecast) / i for i in actual]) / len(actual) * 100


def solution_2(grouped: pd.DataFrame) -> pd.Series:
    def wrapper(values: np.ndarray, index: np.ndarray) -> float:  # noqa
        return mape_2(values, values.mean())

    return grouped.agg(wrapper, engine='numba')


%timeit solution_2(grouped_df)

518 ms ± 20.7 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [107]:
def mape_3(actual, forecast) -> float:  # почему ты считаешь медленнее всех, так ещё и ответ не как у всех?
    return np.mean(np.fabs(actual - forecast) / actual) * 100


def solution_3(grouped: pd.DataFrame) -> pd.Series:
    def wrapper(s: pd.Series) -> float:
        return mape_3(s.to_numpy(), s.mean())

    return grouped.agg(wrapper)


%timeit solution_3(grouped_df)

2.41 s ± 54.9 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [108]:
@jit(nopython=True)
def mape_4(actual, forecast) -> float:
    return np.mean(np.fabs(actual - forecast) / actual) * 100


def solution_4(grouped: pd.DataFrame) -> pd.Series:
    def wrapper(values: np.ndarray, index: np.ndarray) -> float:  # noqa
        return mape_4(values, values.mean())

    return grouped.agg(wrapper, engine='numba')


%timeit solution_4(grouped_df)

497 ms ± 18.6 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [109]:
result_1 = solution_1(grouped_df)
result_2 = solution_2(grouped_df)
result_3 = solution_3(grouped_df)
result_4 = solution_4(grouped_df)

In [110]:
assert np.allclose(result_1, result_2) and np.allclose(result_2, result_3) and np.allclose(result_3, result_4)